# Compare SK Fits — tutorial notebook

This notebook mirrors the functionality of `examples/scripts/compare_sk_fits.py`,
but uses **fixed, editable parameters** instead of argparse and follows the
simplified example style (headless-friendly, `mode='auto3'`).

**What it does:**
- Simulates many SK samples from Gamma parent by accumulating $M$ values.
- Computes SK via `pygsk.core.get_sk(..., N=, d=)`.
- Computes thresholds using `compute_sk_thresholds(..., mode='auto3')` and (optionally) an explicit family.
- Plots a histogram with threshold gates and per-side exceedance rates.


In [ ]:
# --- Parameters (edit here) ---
M = 128
N = 64
d = 1.0
PFA = 1e-3
N_EST = 50_000
SEED = 0
FAMILY = None   # set to 'I', 'III', 'IV', or 'VI' to overlay an explicit-family gate
FIGDIR = "_figs"
PNG_OUT = f"{FIGDIR}/nb_compare_sk_fits_hist.png"

import os, numpy as np
os.makedirs(FIGDIR, exist_ok=True)


In [ ]:
# --- Imports ---
from pygsk.core import get_sk
from pygsk import thresholds
import matplotlib.pyplot as plt


In [ ]:
# --- Simulation: s1, s2 from a Gamma(N, d) parent ---
rng = np.random.default_rng(SEED)
x = rng.gamma(shape=N, scale=d, size=(N_EST, M))
s1 = x.sum(axis=1)
s2 = (x**2).sum(axis=1)

# Compute SK (keyword-only N,d)
sk = get_sk(s1, s2, M, N=N, d=d)
sk_min, sk_max = float(np.min(sk)), float(np.max(sk))
sk_min, sk_max


In [ ]:
# --- Thresholds: auto3 (and optional explicit family) ---
lo_auto, hi_auto, meta_auto = thresholds.compute_sk_thresholds(M, N, d, pfa=PFA, mode="auto3")
rate_lo_auto = float(np.mean(sk < lo_auto))
rate_hi_auto = float(np.mean(sk > hi_auto))

lo_exp = hi_exp = None
rate_lo_exp = rate_hi_exp = None
meta_exp = {}
if FAMILY is not None:
    lo_exp, hi_exp, meta_exp = thresholds.compute_sk_thresholds(M, N, d, pfa=PFA, mode="explicit", family=FAMILY)
    rate_lo_exp = float(np.mean(sk < lo_exp))
    rate_hi_exp = float(np.mean(sk > hi_exp))

print("=== compare_sk_fits (notebook) ===")
print(f"M={M} N={N} d={d} pfa={PFA} n_est={N_EST}")
print(f"Auto thresholds   : lower={lo_auto:.6g} upper={hi_auto:.6g} meta={meta_auto}")
print(f"  Flag rates      : lower={rate_lo_auto:.3e} upper={rate_hi_auto:.3e}")
if lo_exp is not None:
    print(f"Explicit({FAMILY}) : lower={lo_exp:.6g} upper={hi_exp:.6g} meta={meta_exp}")
    print(f"  Flag rates            : lower={rate_lo_exp:.3e} upper={rate_hi_exp:.3e}")


In [ ]:
# --- Plot: histogram with gates + inset ---
fig, ax = plt.subplots()
bins = np.linspace(max(0.1, sk.min()), min(3.0, sk.max()), 400)
ax.hist(sk, bins=bins, density=True, alpha=0.6, label="SK samples")

ax.axvline(lo_auto, linestyle="--", label=f"auto lower {lo_auto:.3g} (p={rate_lo_auto:.2e})")
ax.axvline(hi_auto, linestyle="--", label=f"auto upper {hi_auto:.3g} (p={rate_hi_auto:.2e})")

if lo_exp is not None:
    ax.axvline(lo_exp, linestyle=":", label=f"{FAMILY} lower {lo_exp:.3g} (p={rate_lo_exp:.2e})")
    ax.axvline(hi_exp, linestyle=":", label=f"{FAMILY} upper {hi_exp:.3g} (p={rate_hi_exp:.2e})")

ax.set_xlabel("SK")
ax.set_ylabel("PDF (empirical)")
title = "SK histogram with threshold gates (auto3"
if FAMILY: title += f" + {FAMILY}"
title += ")"
ax.set_title(title)
ax.legend()

inset_txt = f"M={M}  N={N}  d={d}\n" + f"pfa={PFA:g}  n_est={N_EST}"
ax.text(0.02, 0.98, inset_txt, transform=ax.transAxes, va="top",
        fontsize=9, bbox=dict(facecolor="white", alpha=0.85, edgecolor="none"))

fig.savefig(PNG_OUT, dpi=150, bbox_inches="tight")
print("Saved", PNG_OUT)
plt.show()
